In [1]:
import pandas as pd
import numpy as np

In [ ]:
test=pd.read_csv()

In [ ]:
train=pd.read_csv()

In [ ]:
data=pd.concat([train,test],axis=0)

In [ ]:
data.isnull().sum()

In [ ]:
categorical =data.select_dtypes(include=['object']).columns
numeric=data.select_dtypes(include=np.number).columns

In [ ]:
## Categorical _null values

In [ ]:
# Use KNN imputer for more sophisticated imputation (available in the sklearn library).

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df[['categorical_column']] = imputer.fit_transform(df[['categorical_column']])

## Replace it with the constant value
df['categorical_column'] = df['categorical_column'].fillna('Missing')

# Replace null values with the most frequent value (mode) of the column.
df['category'].fillna(df['category'].mode()[0], inplace=True)
print(df)

# Replace null values with a constant value, such as 'Unknown' or 'Missing'.
df['categorical_column'].fillna('Unknown', inplace=True)

In [ ]:
## Numeric null values

In [ ]:
# Mean Imputation
df['numeric_column'].fillna(df['numeric_column'].mean(), inplace=True)

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.drop_duplicates(inplace=True)


In [ ]:
# Dealing with outliers

In [ ]:
Q1=df.quantile(0.25)
Q3=df.quantile(0.75)
IQR=Q3-Q1
outliers=((df<(Q1-1.5*IQR))| (df>(Q3+1.5*IQR))).sum()
outliers
# Replace outliers with median
df = df.mask(outliers, df.median(), axis=0)

In [ ]:
# Skewness of data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox, skew


# Assuming 'data' is your DataFrame and you have already handled NaN and duplicates
numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()

# Function to handle skewness and apply Box-Cox transformation
def handle_skewness(data, numeric_cols, skew_threshold=0.5):
    skewed_cols = []

    for col in numeric_cols:
        # Calculate skewness
        col_skewness = skew(data[col])
        
        # Check if the skewness is significant
        if abs(col_skewness) > skew_threshold:
            skewed_cols.append(col)
            if (data[col] > 0).all():
                # Apply Box-Cox transformation
                data[col], _ = boxcox(data[col])
            else:
                # Handle non-positive values by shifting the data
                min_value = data[col].min()
                data[col] = data[col] - min_value + 1
                data[col], _ = boxcox(data[col])
    
    return data, skewed_cols

In [ ]:
# One hot encoding

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Label encoding for ordinal categorical variables
label_encoder = LabelEncoder()
for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])

# One-hot encoding for nominal categorical variables
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [ ]:
# Feature Scaling
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

In [ ]:
# Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
X=df.drop('SalePrice',axis=1)
y=df['SalePrice']

best_features=SelectKBest(score_func=f_regression,k=10).fit(X,y)
feature_names=list(X.columns)
for feature,score in zip(feature_names,best_features.scores_):
    print(feature,score)
selected_indices=best_features.get_support(indices=True)
selected_features=[feature_names[i] for i in selected_indices]

In [ ]:
# Model building

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))